In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# !pip install py-hanspell
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install soynlp
!pip install nltk
!pip install konlpy
!pip install haversine
!pip install transformers
!pip install sentencepiece
!pip install tqdm
# !pip install fasttext
# !pip install emoji transformers hanspell
!pip install emoji

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-ecnmms0s
  Running command git clone --filter=blob:none --quiet https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-ecnmms0s
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit b32a889cbd10b006d2f4aba118f0cd5b677e2979
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 29.3 MB/s eta 0:00:00
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2286919 sha256=6e8cbb5cce4958cf6099a87c1a19df5abaf5c72541113ff12d480af1b7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 2.4 MB/s eta 0:00:00


# **데이터 불러오기**

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/big_project/df_positive_keyword.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   콘텐츠명                1566 non-null   object 
 1   구명                  1566 non-null   object 
 2   새주소[도로명 주소]         1566 non-null   object 
 3   지번주소                857 non-null    object 
 4   좌표[X]               1566 non-null   float64
 5   좌표[Y]               1566 non-null   float64
 6   전화번호                1433 non-null   object 
 7   주요시설                28 non-null     object 
 8   중분류                 1566 non-null   object 
 9   대분류                 1566 non-null   object 
 10  실내_실외               1566 non-null   object 
 11  시군                  602 non-null    object 
 12  키워드                 596 non-null    object 
 13  웹URL                926 non-null    object 
 14  키즈메뉴(아이가먹을수있는 메뉴)   569 non-null    object 
 15  유아용품                570 non-null    object 
 16  영업시간  

# **카카오리뷰 / 네이버리뷰 ------합치기**

In [ ]:
def parse_reviews(review_str):
    if review_str in ['검색불가', '리뷰없음', '방문자 리뷰 없음']:
        return []
    try:
        reviews = eval(review_str)
        return reviews
    except:
        return []
#검색불가, 리뷰없음, 방문자 리뷰 없음 ---> 빈리스트로 처리
df['kakao_review_e'] = df['kakao_review'].apply(parse_reviews)
df['naver_review_e'] = df['naver_review'].apply(parse_reviews)

#문자열에서 리스트로 변환한후 'total_review'에 합치기 ---> 최종 리뷰!!(카카오 + 네이버)
df['total_review'] = df['kakao_review_e'] + df['naver_review_e']

# **감정분석 모델 불러오기**

In [ ]:
model_name = 'kykim/bert-kor-base'
import torch
from transformers import BertTokenizerFast
from torch.utils.data import Dataset, DataLoader

class Tokenizing(Dataset):
  # init 함수 생성
  def __init__(self, data, model_name):
    self.data = data # document, label로 구성된 dataframe 전달
    self.tokenizer = BertTokenizerFast.from_pretrained(model_name) # Huggingface 토크나이저 생성

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sentence = self.data.iloc[idx]['document'] #iloc : dataframe 정수기반 위치 인덱싱
    label = self.data.iloc[idx]['label']

    # 토크나이징
    tokens = self.tokenizer(
        sentence,               # 1개 문장
        return_tensors='pt',    # 텐서로 반환
        truncation=True,        # 잘라내기 적용
        padding='max_length',   # padding 적용
        add_special_tokens=True # 스페셜 토큰 적용
    )

    input_ids = tokens['input_ids'].squeeze(0)          #2D->1D
    attention_mask = tokens['attention_mask'].squeeze(0)#2D->1D
    token_type_ids = torch.zeros_like(attention_mask) # 단일문장 처리
    # BERT는 두개 문장 입력시 0과1로 구분함. 위 코드로 어텐션마스크 크기만큼의 0텐서를 생성하여 단일문장만 처리하도록함.

    return {
        'input_ids' : input_ids,
        'attention_mask' : attention_mask,
        'token_type_ids' : token_type_ids,
    }, torch.tensor(label)

from transformers import BertModel
import torch
import torch.nn as nn
import torch.optim as optim

class CustomBert(nn.Module):
  def __init__(self, model_name, dropout_rate=0.5):
    # 부모 클래스 초기화
    super(CustomBert, self).__init__()
    # Pre-Trained 모델 지정
    self.bert = BertModel.from_pretrained(model_name)
    # dropout 설정하기
    self.dr = nn.Dropout(p=dropout_rate)
    # 최종 출력층 정의 (768,2)
    # 768은 BERT모델의 'hidden_size'이며, 모델의 마지막 은닉층에서 나오는 특징 벡터의 차원수이다.
    # 2는 최종적으로 분류하고자하는 클래스의 수를 의미한다.
    self.fc = nn.Linear(768, 2)

  def forward(self, input_ids, attention_mask, token_type_ids):
    # 입력을 pre-trained bert model 로 대입
    output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # 결과의 last_hidden_state 가져옴
    last_hidden_state = output['last_hidden_state']
    # last_hidden_state[:, 0, :]는 [CLS] 토큰을 가져옴
    x = self.dr(last_hidden_state[:, 0, :])
    # FC 을 거쳐 최종 출력
    x = self.fc(x)

    return x

# CustomBert 모델 만들기
bert = CustomBert(model_name)
# bert.to(device)
bert.cuda()

import torch.nn.functional as F

class CustomPredictor():
  def __init__(self, model, tokenizer, labels:dict):
    self.model = model
    self.tokenizer = tokenizer
    self.labels = labels

  def predict(self, sentence):
    tokens = self.tokenizer(
        sentence,
        return_tensors='pt',
        truncation=True,
        padding='max_length',
        add_special_tokens=True
    )
    tokens = {k:v.cuda() for k,v in tokens.items()}
    # tokens.to(device)
    # tokens.cuda()
    # prediction = self.model(**tokens)
    # 모델 예측 수행
    logits = self.model(**tokens)
    # print(prediction) tensor([[ 2.9543, -3.3106]], device='cuda:0', grad_fn=<AddmmBackward0>)
    prediction = F.softmax(logits, dim=1)
    # print(prediction) tensor([[0.9981, 0.0019]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
    output = prediction.argmax(dim=1).item()
    # print(output)      0
    prob, result = prediction.max(dim=1)[0].item(), self.labels[output]
    # print(f'[{result}] -- 확률은: {prob*100:.3f}% 입니다.')
    # print(prob)        0.9981016516685486
    #print(result)      #부정 리뷰 입니다.
    positivity = prediction[0][1].item()
    # print(positivity)
    return output, positivity

In [ ]:
model_path = '/content/drive/MyDrive/big_project/customBert.pth'  # 모델 가중치 파일 경로 설정

# 저장한 state_dict를 bert 모델에 load -> 진짜 내 모델!!
bert.load_state_dict(torch.load(model_path))

tokenizer = BertTokenizerFast.from_pretrained(model_name)

labels = {
    0: '부정 리뷰 입니다.',
    1: '긍정 리뷰 입니다.'
}

predictor = CustomPredictor(bert, tokenizer, labels)

# **리뷰 전처리 작업(맞춤법 교정)**

In [ ]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline

import re
from pykospacing import Spacing # 띄어쓰기 교정
from soynlp.normalizer import emoticon_normalize  # 반복 단어 제거
from konlpy.tag import Okt  # 품사 태깅
from emoji import demojize
import emoji

In [ ]:
# 맞춤법 교정 모델 text2text GAN (Pretrained Language Model : PLM 모델 활용)
corrector_model = T5ForConditionalGeneration.from_pretrained('j5ng/et5-typos-corrector')
corrector_tokenizer = T5Tokenizer.from_pretrained('j5ng/et5-typos-corrector')

typos_corrector = pipeline(
    "text2text-generation",
    model=corrector_model,
    tokenizer=corrector_tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    framework="pt",
)

# 맞춤법 교정
def spelling_check(text):
    """
    맞춤법 교정
    """
    # 텍스트를 문장 단위로 분할합니다.
    parts_of_text = re.split(r'([,.!?])', text)  # 구분자를 유지하기 위해 그룹화

    correct_text = []

    for i in range(0, len(parts_of_text), 2):
        sentence = parts_of_text[i]
        if sentence.strip():  # 문장이 비어있지 않은 경우에만 처리
            # 맞춤법 교정을 적용합니다.
            corrected = typos_corrector("맞춤법을 고쳐주세요: " + sentence,
                                        max_length=128,
                                        num_beams=5,
                                        early_stopping=True)[0]['generated_text']
            corrected = corrected.rstrip('.,!?')
            correct_text.append(corrected)

        # 구분자(특수문자)를 다시 추가합니다.
        if i + 1 < len(parts_of_text):
            correct_text.append(parts_of_text[i + 1])

    return ''.join(correct_text)

# 이모티콘 제거 함수
def remove_emojis_keep_punctuation(text):
    """
    이모티콘을 제거하고 구두점을 유지하는 함수
    """
    text = emoji.replace_emoji(text, replace='')  # 이모티콘 제거
    text = re.sub(r'[^\w\s.,!?]', '', text)  # 이모티콘을 제외한 특수문자 제거
    return text

# 띄어쓰기 처리
def spacing_processing(text):
    """
    띄어쓰기 처리
    """

    spacing = Spacing()
    text = spacing(text)
    return text

# 반복 문자 처리
def repetitive_character_processing(text):
    """
    반복 문자 처리
    """
    text = emoticon_normalize(text, num_repeats=2)
    return text


# 사용자 입력 데이터 처리
def input_process_text(text):
    if not text.strip():  # 빈 문자열일 경우 그대로 반환
        return ""
    text = repetitive_character_processing(text)
    text = remove_emojis_keep_punctuation(text)
    # text = special_character_processing(text)
    text = spacing_processing(text)
    text = spelling_check(text)
    return text

config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
df['total_clean_review'] = ''

In [ ]:
# 문자열 처리를 통해 개별 리뷰 추출

for i in range(len(df)):
    reviews_list = df['total_review'].loc[i]

    # 클린 리뷰 리스트 생성
    cleaned_reviews = [input_process_text(review) for review in reviews_list if review.strip()]


    cleaned_list = list(filter(lambda x: x != '', cleaned_reviews)) # 빈텍스트 처리
    # 텍스트가 없이 느낌표, 점, 물음표만 있는 항목 제거
    cleaned_list = [item for item in cleaned_list if not all(char in ['!', '.', '?'] for char in item)] # 문자로만 이루어져있는 리뷰들 처리

    df['total_clean_review'].loc[i] = cleaned_list

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-70-49b0c6eccf89>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_clean_review'].loc[i] = cleaned_list
<ipython-input-70-49b0c6eccf89>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_clean_review'].loc[i] = cleaned_list
<ipython-input-70-49b0c6eccf89>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_clean_review'].loc[i] = cleaned_list
<ipython-input-7

# **긍/부정 분류**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/big_project/df_positive_keyword.csv')

In [ ]:
# 문자열 리스트형식으로 변환(최종적)
import ast
for i in range(len(data)):
    data['total_clean_review'].loc[i] = ast.literal_eval(data['total_clean_review'].loc[i])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-114-8b4398b2a256>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_clean_review'].loc[i] = ast.literal_eval(data['total_clean_review'].loc[i])
<ipython-input-114-8b4398b2a256>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['total_clean_review'].loc[i] = ast.literal_eval(data['total_clean_review'].loc[i])
<ipython-input-114-8b4398b2a256>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi

In [ ]:
# data에 긍정리뷰 / 부정리뷰 / ai평점  ----> 컬럼 추가하기
data['positive_review'] = ''
data['negative_review'] = ''
data['ai_score'] = ''

In [ ]:
for i in range(len(data)):
    good = [] # 긍정리뷰
    bad = [] # 부정리뷰
    p = 0  # 긍부정 점수
    outputs = []
    cleaned_list = data['total_clean_review'].loc[i]
    # 개별 리뷰 출력
    for idx, review in enumerate(cleaned_list, start=1):
        text = review

        label, positivity = predictor.predict(text)
        p += positivity

        if label == 1:
            good.append(review)
        else:
            bad.append(review)


    data['positive_review'].loc[i] = good
    data['negative_review'].loc[i] = bad
    if len(cleaned_list) != 0:
        data['ai_score'].loc[i] = p/len(cleaned_list)
    else:
        data['ai_score'].loc[i] = 0


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-116-d775949f23e4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ai_score'].loc[i] = p/len(cleaned_list)
<ipython-input-116-d775949f23e4>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['positive_review'].loc[i] = good
<ipython-input-116-d775949f23e4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['negative_review'].loc[i] = bad
<ipython-input-116-d775949f23e4>:

# **긍정리뷰 / 부정리뷰 토큰화**

**긍정 키워드**

In [ ]:
import re
from konlpy.tag import Okt
from nltk.corpus import stopwords
import spacy
from gensim.models import FastText

# Konlpy
okt = Okt()

# 불용어처리
stopwords = ['한테', '모레', '로', '나', '있다', '이미', '많이', '하다', '것', '종일', '매우', '하며', '있는', '곳', '저', '그것', '내일', '과', '그렇다면', '너',
             '누구의', '언제', '있고', '에서', '무엇', '정말', '와', '혹은', '이따가', '가', '그녀', '어제', '진짜', '실제로', '무슨', '만일', '가장', '등', '몹시',
             '께', '하지만', '을', '아주', '이', '누구', '그리고', '에게서', '잠깐', '모두', '에', '으로', '위한', '너무', '여기', '어디', '그러나', '의', '계속', '누가',
             '또는', '한참', '하고', '한테서', '우리', '이것', '까지', '조금', '그', '오늘', '적게', '그렇지만', '만약', '수', '를', '부터', '무엇이', '그래서', '는',
             '및', '방금', '은', '등등', '그런데', '거기', '어떤', '에게', '도', '벌써', '당신', '지금', '다', '저것', '할', '그냥', '사실', '어느', '얼마나', '읽을',
             '잠시', '어떻게', '한', '금방', '저기', '가득', '먹다', '맛', '가다', '오다', '자다', '같다', '더', '넘다', '또', '이다', '아쉽다', '바', '꼭', '보다', '되어다', '나오다',
             '요', '들다', '처', '다음', '때', '되다', '거', '싶다', '점', '주다', '리지', '널', '크리스', '크림', '피', '드', '안', '역시', '받다', '글레이', '좀', '꾸다', '지점', '해',
             '덮다', '집', '굿', '시', '비', '없다', '나다', '생기다', '바로', '못', '날']

# 한글이 아닌 특수문자 제거
def extract_keywords(text):
    # 특수문자, 기호, 숫자 제거
    # text = re.sub('[^ ㄱ-ㅣ가-힣]', '', text)
    # parts_of_text = text.split(', ')
    parts_of_text = re.split(r'([,.!?])', text)
    # print(parts_of_text)
    processed_tokens = []

    for part in parts_of_text:
        # print(part)
        tokens = okt.pos(part.strip(), stem=True)
        # print(tokens)
        i = 0
        while i < len(tokens):
            token = tokens[i]
            # print(f'{i}', token)
            # 명사, 형용사, 동사인 경우에만 처리
            if token[1] in ['Noun', 'Adjective', 'Verb'] and token[0] not in stopwords:
                processed_tokens.append(token[0])

                # 명사이거나 형용사인 경우 -> 명사 + 명사 : 복합명사 ex) 숲체험 / 형용사 + 명사 : 꾸며주는 명사 ex) 친절한 직원
                if token[1] in ['Noun', 'Adjective']:
                    # 다음 토큰도 명사이고, 두 토큰을 합쳐서 하나의 토큰으로 처리
                    if i < len(tokens) - 1 and tokens[i + 1][1] in ['Noun'] and tokens[i + 1][0] not in stopwords:
                        combined_token = token[0] + tokens[i + 1][0]
                        processed_tokens.append(combined_token)
                        # 형용사 + 명사 + 명사 또는 명사 + 명사 + 명사 인 경우 앞에 두개의 토큰을 더한 값은 삭제 후 새로운 토큰 삽입
                        # ex) 넓은 키즈카페, ex)문화예술회관
                        if i < len(tokens) - 2 and tokens[i + 2][1] in ['Noun'] and tokens[i + 2][0] not in stopwords:
                            combined_token = token[0] + tokens[i + 1][0] + tokens[i + 2][0]
                            processed_tokens.pop(-1)
                            processed_tokens.append(combined_token)

                # 부정 표현 처리
                if i < len(tokens) - 1 and tokens[i + 1][0] == '않다':
                    combined_token = token[0] + '않다'
                    processed_tokens.pop(-1)  # 이전 토큰 제거
                    processed_tokens.append(combined_token)
                    i += 1  # '않다' 토큰 건너뛰기
            i += 1
    return processed_tokens

In [ ]:
# 데이터에 긍정키워드 컬럼 추가
data['positive_keyword'] = ''

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   콘텐츠명                        1566 non-null   object 
 1   구명                          1566 non-null   object 
 2   새주소[도로명 주소]                 1566 non-null   object 
 3   지번주소                        857 non-null    object 
 4   좌표[X]                       1566 non-null   float64
 5   좌표[Y]                       1566 non-null   float64
 6   전화번호                        1433 non-null   object 
 7   주요시설                        28 non-null     object 
 8   중분류                         1566 non-null   object 
 9   대분류                         1566 non-null   object 
 10  실내_실외                       1566 non-null   object 
 11  시군                          602 non-null    object 
 12  키워드                         596 non-null    object 
 13  웹URL                        926 n

In [ ]:
from itertools import chain

for i in range(len(data)):
    rev = data['positive_review'].loc[i]

    keyword = list(chain.from_iterable(extract_keywords(text) for text in rev))



    from collections import Counter

    # 키워드 빈도수 계산
    keyword_counts = Counter(keyword)
    # 상위 15개 키워드 추출
    top_15_keywords = keyword_counts.most_common(15)

    #상위 50개 전체 키워드
    top_nouns = dict(keyword_counts.most_common(50))

    # 적어도 3번 이상 나온 키워드 필터링
    top_15_keywords_over_3 = [kw[0] for kw in top_15_keywords if kw[1] >= 3]

    data['positive_keyword'].loc[i] = top_15_keywords_over_3 #긍정키워드 추가


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-312-4d5252e3f356>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['positive_keyword'].loc[i] = top_5_keywords_over_3
<ipython-input-312-4d5252e3f356>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['positive_keyword'].loc[i] = top_5_keywords_over_3
<ipython-input-312-4d5252e3f356>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['positive_keyword'].loc[i] = top_5_ke

In [ ]:
data['positive_keyword'].loc[1564]

['좋다',
 '체험',
 '떡',
 '아이',
 '만들기',
 '좋아하다',
 '떡만들기체험',
 '만들기체험',
 '재미있다',
 '떡체험',
 '설명',
 '시간',
 '전',
 '게',
 '떡만들기']

# **워드클라우드**

In [ ]:
!pip install wordcloud

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import numpy as np
from PIL import Image

In [ ]:
# 워드클라우드에 들어갈 키워드 컬럼 추가
# data['wordcloud_positive_keyword'] = ''
data['wordcloud_positive_keyword'] = [{}] * len(df)  # 초기화

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1566 entries, 0 to 1565
Data columns (total 43 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   콘텐츠명                        1566 non-null   object 
 1   구명                          1566 non-null   object 
 2   새주소[도로명 주소]                 1566 non-null   object 
 3   지번주소                        857 non-null    object 
 4   좌표[X]                       1566 non-null   float64
 5   좌표[Y]                       1566 non-null   float64
 6   전화번호                        1433 non-null   object 
 7   주요시설                        28 non-null     object 
 8   중분류                         1566 non-null   object 
 9   대분류                         1566 non-null   object 
 10  실내_실외                       1566 non-null   object 
 11  시군                          602 non-null    object 
 12  키워드                         596 non-null    object 
 13  웹URL                        926 n

워드클라우드 긍정키워드 추가하기

In [ ]:
from itertools import chain

for i in range(len(data)):
    rev = data['positive_review'].loc[i]

    keyword = list(chain.from_iterable(extract_keywords(text) for text in rev))



    from collections import Counter

    # 키워드 빈도수 계산
    keyword_counts = Counter(keyword)
    #상위 50개 전체 키워드
    top_nouns = dict(keyword_counts.most_common(50))

    data['wordcloud_positive_keyword'].loc[i] = top_nouns #긍정키워드 추가


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-23-574807e2c224>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['wordcloud_positive_keyword'].loc[i] = top_nouns #긍정키워드 추가
<ipython-input-23-574807e2c224>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['wordcloud_positive_keyword'].loc[i] = top_nouns #긍정키워드 추가
<ipython-input-23-574807e2c224>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['wordcloud_positive_keyw

In [ ]:
data['wordcloud_positive_keyword'].loc[500]

array([134,  59,  58,  53,  46,  45,  43,  42,  35,  35,  35,  26,  25,
        24,  22,  22,  22,  21,  20,  19,  19,  17,  17,  17,  16,  16,
        16,  15,  13,  13,  13,  13,  12,  12,  12,  12,  12,  11,  11,
        11,  11,  10,  10,  10,   9,   9,   9,   9,   9,   9])

기본 모양

In [ ]:
wc = WordCloud (max_words=50,
           background_color='white',
           font_path= '/content/drive/MyDrive/big_project/NanumSquareR.ttf'
           )
wc.generate_from_frequencies(data['wordcloud_positive_keyword'].loc[456])
plt.figure(figsize=(8,8))
plt.imshow(wc)
plt.axis('off')
# plt.savefig('wordcloud.png') #그림을 저장
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'items'

어린이 모양

In [ ]:
wc = WordCloud (max_words=100,
           background_color='black',
           mask=np.array(Image.open(r"/content/drive/MyDrive/big_project/kids.png")), #마스크 추가
           font_path= '/content/drive/MyDrive/big_project/NanumSquareR.ttf',
           colormap='autumn',
           )
wc.generate_from_frequencies(top_nouns)
plt.figure(figsize=(8,8))
plt.imshow(wc)
plt.axis('off')
plt.savefig('wordcloud.png') #그림을 저장
plt.show()

애기 모양

In [ ]:
# mask = np.array(Image.open(r"/content/drive/MyDrive/big_project/kids3.png"))

# 마스크 이미지 불러오기 및 해상도 줄이기
mask_image = Image.open(r"/content/drive/MyDrive/big_project/kids3.png")
mask_image = mask_image.resize((mask_image.width // 2, mask_image.height // 2), Image.ANTIALIAS)
mask = np.array(mask_image)

wc = WordCloud (max_words=100,
           background_color='black',
           mask=mask, #마스크 추가
           font_path= '/content/drive/MyDrive/big_project/NanumSquareR.ttf',
           colormap='rainbow',
           scale=1,
           )
wc.generate_from_frequencies(top_nouns)
plt.figure(figsize=(8,8))
plt.imshow(wc)
plt.axis('off')
plt.savefig('wordcloud.png') #그림을 저장
plt.show()